# Part 2: Basic LLM Chat Tool

## Introduction

In this part, you'll create a simple command-line chat tool that interacts with a Large Language Model (LLM) through the Hugging Face API. This tool will allow you to have conversations with an LLM about healthcare topics.

## Learning Objectives

- Connect to the Hugging Face API
- Create a basic interactive chat loop
- Handle simple error cases
- Test with healthcare questions

## Setup and Installation

In [ ]:
# Install required packages
%pip install -r requirements.txt

# Additional packages for LLM API interaction
%pip install requests

# Import necessary libraries
import os
import sys
import requests
import time
import logging
import argparse
from typing import Optional

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Create directories
os.makedirs('utils', exist_ok=True)
os.makedirs('results/part_2', exist_ok=True)

os.environ["HUGGINGFACE_API_KEY"] = "YOUR_API_KEY_HERE"
api_key = os.environ.get("HUGGINGFACE_API_KEY")
if api_key:
    print("API key successfully set in environment variables")
else:
    print("Failed to set API key in environment")

# Create headers for API requests
headers = {"Authorization": f"Bearer {api_key}"}

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


API key successfully set in environment variables


## 1. Connecting to the Hugging Face API

The Hugging Face Inference API provides access to many language models. We'll use models that are available on the free tier.

In [2]:
# Example of a simple API request to Hugging Face
API_URL = API_URL = "https://api-inference.huggingface.co/models/facebook/bart-large-cnn"

def query(payload):
    try:
        # Get API key from environment
        api_key = os.environ.get("HUGGINGFACE_API_KEY")
        if not api_key:
            return {"error": "API key not found in environment variables"}
            
        # Set up headers with the API key
        headers = {"Authorization": f"Bearer {api_key}"}
        
        # Send request to the API
        response = requests.post(API_URL, headers=headers, json=payload)
        
        # Check if the request was successful
        response.raise_for_status()
        
        # Parse and return the JSON response
        return response.json()
        
    except requests.exceptions.RequestException as e:
        logger.error(f"API request failed: {e}")
        # Check if we got a response before the exception
        if 'response' in locals():
            logger.error(f"Response status code: {response.status_code}")
            logger.error(f"Response content: {response.text}")
        return {"error": str(e)}
    except Exception as e:
        logger.error(f"Unexpected error: {e}")
        return {"error": str(e)}
    
# Test the query function
test_payload = {"inputs": "What are the symptoms of diabetes?"}
print("Sending test query...")
response = query(test_payload)
print("API Response:")
print(response)

Sending test query...
API Response:
[{'summary_text': 'CNN.com will feature iReporter photos in a weekly Travel Snapshots gallery. Visit CNN.com/Travel each week for a new gallery of snapshots. This week, we feature snapshots from the U.S. and Canada. See www.dailymail.co.uk/Travel.'}]


## 2. Creating Simple Chat Scripts

Your task is to create two simple scripts that interact with the Hugging Face API:

1. A basic one-off chat script (`utils/one_off_chat.py`)
2. A contextual conversation script (`utils/conversation.py`)

### One-Off Chat Script

Create a script that handles independent interactions (each prompt/response is separate):

In [3]:
# utils/one_off_chat.py

import requests
import argparse
import os
import logging
import time  # Added import for sleep

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

def get_response(prompt, model_name="facebook/bart-large-cnn", api_key=None, history=None, history_length=3, max_retries=3):
    # Get API key from environment if not provided
    if api_key is None:
        api_key = os.environ.get("HUGGINGFACE_API_KEY")
        if not api_key:
            logger.error("No API key provided and none found in environment")
            return "Error: API key is required but none was provided"
    
    # Set up the API URL and headers
    api_url = f"https://api-inference.huggingface.co/models/{model_name}"
    headers = {"Authorization": f"Bearer {api_key}"}
    
    # Create payload with the prompt
    payload = {"inputs": prompt}
    
    # Initialize retry counter and backoff time
    retry_count = 0
    backoff_time = 1  # Start with 1 second
    
    while retry_count <= max_retries:
        try:
            # Send request to the API
            response = requests.post(api_url, headers=headers, json=payload)
            
            # Check for rate limiting
            if response.status_code == 429:
                retry_count += 1
                if retry_count > max_retries:
                    logger.warning(f"Rate limit exceeded after {max_retries} retries")
                    return "Error: Rate limit exceeded. Please try again later."
                
                # Get retry-after header if available, otherwise use exponential backoff
                retry_after = response.headers.get('retry-after')
                if retry_after and retry_after.isdigit():
                    wait_time = int(retry_after)
                else:
                    wait_time = backoff_time
                    backoff_time *= 2  # Exponential backoff
                
                logger.info(f"Rate limited. Retrying in {wait_time} seconds (attempt {retry_count}/{max_retries})")
                time.sleep(wait_time)
                continue  # Try again
            
            # For non-rate-limiting errors, raise the exception
            response.raise_for_status()
            
            # Parse the response
            result = response.json()
            
            # Extract the generated text from the response - updated to handle BART format
            if isinstance(result, list) and len(result) > 0:
                if isinstance(result[0], dict):
                    # Handle different model response formats
                    if "summary_text" in result[0]:
                        return result[0]["summary_text"]
                    elif "generated_text" in result[0]:
                        return result[0]["generated_text"]
                elif isinstance(result[0], str):
                    return result[0]
            
            # If we get here, the response format wasn't recognized
            logger.warning(f"Unexpected response format: {result}")
            return str(result)
            
        except requests.exceptions.RequestException as e:
            logger.error(f"API request failed: {e}")
            if 'response' in locals():
                logger.error(f"Response status code: {response.status_code}")
                logger.error(f"Response content: {response.text}")
            
            # Check if we should retry based on error type
            if isinstance(e, requests.exceptions.ConnectionError) and retry_count < max_retries:
                retry_count += 1
                wait_time = backoff_time
                backoff_time *= 2  # Exponential backoff
                logger.info(f"Connection error. Retrying in {wait_time} seconds (attempt {retry_count}/{max_retries})")
                time.sleep(wait_time)
                continue  # Try again
                
            return f"Error: Failed to get response from the model. {str(e)}"
        
        except Exception as e:
            logger.error(f"Unexpected error: {e}")
            return f"Error: {str(e)}"

def run_chat():
    """Run an interactive chat session"""
    print("Welcome to the Simple LLM Chat! Type 'exit' to quit.")
    
    # Check if API key is available
    api_key = os.environ.get("HUGGINGFACE_API_KEY")
    if not api_key:
        print("Warning: No API key found in environment variables.")
        api_key = input("Please enter your Hugging Face API key: ")
        if not api_key:
            print("No API key provided. Exiting.")
            return
    
    while True:
        user_input = input("\nYou: ")
        if user_input.lower() in ['exit', 'quit', 'bye']:
            print("Goodbye!")
            break
        
        if not user_input.strip():
            print("Bot: Please enter a question or prompt.")
            continue
            
        # Get response from the model
        print("Bot: Thinking...")
        response = get_response(user_input, api_key=api_key)
        print(f"Bot: {response}")
        
def main():
    try:
        # Check if we're running in IPython/Jupyter
        get_ipython
        # We're in a notebook, so don't parse arguments - just run the chat
        run_chat()
        return
    except NameError:
        # We're not in a notebook, parse arguments as normal
        parser = argparse.ArgumentParser(description="Chat with an LLM")
        parser.add_argument("--model", type=str, default="facebook/bart-large-cnn",
                          help="Model to use (default: facebook/bart-large-cnn)")
        parser.add_argument("--key", type=str, help="Hugging Face API key")
        parser.add_argument("--prompt", type=str, help="Single prompt (non-interactive mode)")
        parser.add_argument("--max-retries", type=int, default=3,
                          help="Maximum number of retry attempts for rate limiting (default: 3)")
        
        args = parser.parse_args()
        
        # If a single prompt is provided, just get the response and exit
        if args.prompt:
            response = get_response(args.prompt, model_name=args.model, api_key=args.key, max_retries=args.max_retries)
            print(f"Response: {response}")
        else:
            # Otherwise, run the interactive chat
            run_chat()

# Call the main function when running this script
if __name__ == "__main__":
    main()

Welcome to the Simple LLM Chat! Type 'exit' to quit.
Goodbye!


### Contextual Conversation Script

Create a script that maintains conversation history:

In [4]:
# utils/conversation.py

import requests
import argparse
import os
import logging
import time  # Added import for sleep

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

def get_response(prompt, history=None, model_name="facebook/bart-large-cnn", api_key=None, history_length=3, max_retries=3):
    # Initialize history if None
    if history is None:
        history = []
    
    # Get API key from environment if not provided
    if api_key is None:
        api_key = os.environ.get("HUGGINGFACE_API_KEY")
        if not api_key:
            logger.error("No API key provided and none found in environment")
            return "Error: API key is required but none was provided"
    
    # Set up the API URL and headers
    api_url = f"https://api-inference.huggingface.co/models/{model_name}"
    headers = {"Authorization": f"Bearer {api_key}"}
    
    # Format a prompt that includes previous exchanges (limited by history_length)
    context = ""
    if history:
        # Take only the most recent exchanges up to history_length
        recent_history = history[-history_length:]
        for i, (past_prompt, past_response) in enumerate(recent_history):
            context += f"Question {i+1}: {past_prompt}\nAnswer {i+1}: {past_response}\n\n"
    
    # Add the current prompt with context
    full_prompt = f"{context}Question: {prompt}\nAnswer:"
    
    # Create payload with the contextual prompt
    payload = {"inputs": full_prompt}
    
    # Initialize retry counter and backoff time
    retry_count = 0
    backoff_time = 1  # Start with 1 second
    
    while retry_count <= max_retries:
        try:
            # Send request to the API
            response = requests.post(api_url, headers=headers, json=payload)
            
            # Check for rate limiting
            if response.status_code == 429:
                retry_count += 1
                if retry_count > max_retries:
                    logger.warning(f"Rate limit exceeded after {max_retries} retries")
                    return "Error: Rate limit exceeded. Please try again later."
                
                # Get retry-after header if available, otherwise use exponential backoff
                retry_after = response.headers.get('retry-after')
                if retry_after and retry_after.isdigit():
                    wait_time = int(retry_after)
                else:
                    wait_time = backoff_time
                    backoff_time *= 2  # Exponential backoff
                
                logger.info(f"Rate limited. Retrying in {wait_time} seconds (attempt {retry_count}/{max_retries})")
                time.sleep(wait_time)
                continue  # Try again
            
            # For non-rate-limiting errors, raise the exception
            response.raise_for_status()
            
            # Parse the response
            result = response.json()
            
            # Extract the generated text from the response - updated to handle BART format
            if isinstance(result, list) and len(result) > 0:
                if isinstance(result[0], dict):
                    # Handle different model response formats
                    if "summary_text" in result[0]:
                        generated_text = result[0]["summary_text"]
                        # Remove the prompt if it's included in the response
                        if full_prompt in generated_text:
                            generated_text = generated_text.replace(full_prompt, "").strip()
                        return generated_text
                    elif "generated_text" in result[0]:
                        generated_text = result[0]["generated_text"]
                        # Remove the prompt if it's included in the response
                        if full_prompt in generated_text:
                            generated_text = generated_text.replace(full_prompt, "").strip()
                        return generated_text
                elif isinstance(result[0], str):
                    return result[0]
            
            logger.warning(f"Unexpected response format: {result}")
            return str(result)
            
        except requests.exceptions.RequestException as e:
            logger.error(f"API request failed: {e}")
            if 'response' in locals():
                logger.error(f"Response status code: {response.status_code}")
                logger.error(f"Response content: {response.text}")
                
            # Check if we should retry based on error type
            if isinstance(e, requests.exceptions.ConnectionError) and retry_count < max_retries:
                retry_count += 1
                wait_time = backoff_time
                backoff_time *= 2  # Exponential backoff
                logger.info(f"Connection error. Retrying in {wait_time} seconds (attempt {retry_count}/{max_retries})")
                time.sleep(wait_time)
                continue  # Try again
                
            return f"Error: Failed to get response from the model. {str(e)}"
        
        except Exception as e:
            logger.error(f"Unexpected error: {e}")
            return f"Error: {str(e)}"

def run_chat():
    """Run an interactive chat session with context"""
    print("Welcome to the Contextual LLM Chat! Type 'exit' to quit.")
    
    # Check if API key is available
    api_key = os.environ.get("HUGGINGFACE_API_KEY")
    if not api_key:
        print("Warning: No API key found in environment variables.")
        api_key = input("Please enter your Hugging Face API key: ")
        if not api_key:
            print("No API key provided. Exiting.")
            return
    
    # Initialize conversation history
    history = []
    
    while True:
        user_input = input("\nYou: ")
        if user_input.lower() in ['exit', 'quit', 'bye']:
            print("Goodbye!")
            break
        
        if not user_input.strip():
            print("Bot: Please enter a question or prompt.")
            continue
            
        # Get response using conversation history
        print("Bot: Thinking...")
        response = get_response(user_input, history=history, api_key=api_key)
        print(f"Bot: {response}")
        
        # Update history with the current exchange
        history.append((user_input, response))
        
def main():
    try:
        # Check if we're running in IPython/Jupyter
        get_ipython
        run_chat()
        return
    except NameError:
        # We're not in a notebook, parse arguments as normal
        parser = argparse.ArgumentParser(description="Chat with an LLM using conversation history")
        parser.add_argument("--model", type=str, default="facebook/bart-large-cnn",
                        help="Model to use (default: facebook/bart-large-cnn)")
        parser.add_argument("--key", type=str, help="Hugging Face API key")
        parser.add_argument("--history-length", type=int, default=3,
                        help="Number of previous exchanges to include in context (default: 3)")
        parser.add_argument("--prompt", type=str, help="Single prompt (non-interactive mode)")
        parser.add_argument("--max-retries", type=int, default=3,
                        help="Maximum number of retry attempts for rate limiting (default: 3)")
        
        args = parser.parse_args()
        
        # If a single prompt is provided, just get the response and exit
        if args.prompt:
            response = get_response(args.prompt, model_name=args.model, api_key=args.key, max_retries=args.max_retries)
            print(f"Response: {response}")
        else:
            # Otherwise, run the interactive chat
            run_chat()
    
if __name__ == "__main__":
    main()

Welcome to the Contextual LLM Chat! Type 'exit' to quit.
Goodbye!


## 3. Testing and Evaluation

Create a script to test your chat implementations with specific healthcare questions.

In [7]:
# utils/test_chat.py

import os
import sys
import logging
from pathlib import Path

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

try:
    # Check if we're in a notebook
    get_ipython
    
    # If we are, set the current directory manually
    notebook_path = os.getcwd()
    utils_dir = os.path.join(notebook_path, 'utils')
    
    # Make sure the utils directory exists
    os.makedirs(utils_dir, exist_ok=True)
    
    # Add it to the path
    if utils_dir not in sys.path:
        sys.path.append(utils_dir)
    
    # Save the modules to files if they don't exist yet
    one_off_path = os.path.join(utils_dir, 'one_off_chat.py')
    conv_path = os.path.join(utils_dir, 'conversation.py')
    
    # Get our functions from the notebook environment
    logger.info("Running in notebook mode, saving modules...")
    
except NameError:
    # We're running as a script
    current_dir = Path(__file__).parent
    sys.path.append(str(current_dir))

# Import our chat modules
try:
    # Try importing as modules
    try:
        from one_off_chat import get_response as get_one_off_response
        from conversation import get_response as get_contextual_response
        logger.info("Successfully imported chat modules")
    except ImportError:
        logger.info("Using functions from notebook environment")
        # We'll use the functions defined in previous cells
        get_one_off_response = get_response  # from one-off chat cell
        get_contextual_response = get_response  # from conversation.py cell
        
except Exception as e:
    logger.error(f"Failed to import chat modules: {e}")
    logger.error("Make sure to run the cells defining get_response functions first!")
    raise


def test_one_off_chat(questions, model_name="facebook/bart-large-cnn", api_key=None):
    results = {}
    
    # Get API key from environment if not provided
    if api_key is None:
        api_key = os.environ.get("HUGGINGFACE_API_KEY")
        if not api_key:
            logger.error("No API key provided and none found in environment")
            return {"error": "API key is required but none was provided"}

    logger.info(f"Testing one-off chat with {len(questions)} questions")
    
    for question in questions:
        logger.info(f"Testing question: {question}")
        # Get response using the one-off chat function
        response = get_one_off_response(question, model_name=model_name, api_key=api_key)
        results[question] = response
        logger.info(f"Got response: {response[:50]}...")  # Log first 50 chars
        
    return results

def test_contextual_chat(questions, model_name="facebook/bart-large-cnn", api_key=None):
    results = []
    history = []
    
    # Get API key from environment if not provided
    if api_key is None:
        api_key = os.environ.get("HUGGINGFACE_API_KEY")
        if not api_key:
            logger.error("No API key provided and none found in environment")
            return [("error", "API key is required but none was provided")]
    
    logger.info(f"Testing contextual chat with {len(questions)} questions")
    
    for question in questions:
        logger.info(f"Testing contextual question: {question}")
        # Get response using the contextual chat function
        response = get_contextual_response(
            question, 
            history=history, 
            model_name=model_name, 
            api_key=api_key
        )
        results.append((question, response))
        logger.info(f"Got response: {response[:50]}...")  # Log first 50 chars
        
        # Update history with the current exchange
        history.append((question, response))
        
    return results

# List of healthcare questions to test
test_questions = [
    "What are the symptoms of gout?",
    "How is gout diagnosed?",
    "What treatments are available for gout?",
    "What lifestyle changes can help manage gout?",
    "What foods should be avoided with gout?"
]

def save_results(results, contextual_results=None, output_file="results/part_2/example.txt"):
    # Ensure the directory exists
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    
    with open(output_file, 'w') as f:
        # Write header
        f.write("# LLM Chat Tool Test Results\n\n")

        # Add medical disclaimer
        f.write("## IMPORTANT MEDICAL DISCLAIMER\n\n")
        f.write("**WARNING: The responses generated by this LLM are NOT medically accurate and should NOT be used for healthcare decisions.**\n\n")
        f.write("The facebook/bart-large-cnn model used was primarily trained to summarize news articles, not to provide medical advice. ")
        f.write("The responses may contain incorrect and incomplete information about symptoms, diagnoses, and treatments. ")
        f.write("Due to using a free language model, not all answers will be available from the website as it is a public site.\n\n")
        f.write("For this assignment, with the free Hugging Face API account, I was only able to get access to the bart-cnn model which is a low-quality language model.\nThe more advanced models (flan-T5, gpt2, PubMedBERT, Bio_ClinicalBERT, etc.) were unable to be accessed (404 errors/permission denied) due to using the free account.\n\n")

        # Write usage examples
        f.write("## Usage Examples\n\n")
        f.write("```bash\n")
        f.write("# Run the one-off chat\n")
        f.write("python utils/one_off_chat.py\n\n")
        f.write("# Run the contextual chat\n")
        f.write("python utils/conversation.py\n")
        f.write("```\n\n")
        
        # Write test results for one-off chat
        f.write("## One-Off Chat Test Results\n\n")
        f.write("```csv\n")
        f.write("question,response\n")
        
        for question, response in results.items():
            # Format the question and response for CSV
            q = question.replace(',', '').replace('\n', ' ')
            r = str(response).replace(',', '').replace('\n', ' ')
            f.write(f"{q},{r}\n")
            
        f.write("```\n\n")
        
        # Write test results for contextual chat if provided
        if contextual_results:
            f.write("## Contextual Chat Test Results\n\n")
            f.write("```\n")
            
            # For each exchange, show the progression of the conversation
            for i, (question, response) in enumerate(contextual_results):
                f.write(f"### Exchange {i+1}\n\n")
                f.write(f"**Q:** {question}\n\n")
                f.write(f"**A:** {response}\n\n")
                
            f.write("```\n")

# Run the test and save results
def main():
    # Get API key from environment
    api_key = os.environ.get("HUGGINGFACE_API_KEY")
    if not api_key:
        logger.error("No API key found in environment variables.")
        sys.exit(1)
    
    model_name = "facebook/bart-large-cnn"  # Use the model that works for you
    
    try:
        # Test the one-off chat
        logger.info("Testing one-off chat...")
        results = test_one_off_chat(test_questions, model_name, api_key)
        
        # Test the contextual chat
        logger.info("Testing contextual chat...")
        contextual_results = test_contextual_chat(test_questions, model_name, api_key)
        
        # Save the results
        save_results(results, contextual_results)
        logger.info(f"Test results saved to results/part_2/example.txt")
        
        # Print summary
        logger.info("Testing complete!")
        logger.info(f"Tested {len(test_questions)} questions")
        logger.info(f"One-off chat responses: {len(results)}")
        logger.info(f"Contextual chat responses: {len(contextual_results)}")
        
    except Exception as e:
        logger.error(f"Error during testing: {e}")
        import traceback
        logger.error(traceback.format_exc())
        sys.exit(1)

if __name__ == "__main__":
    main()

2025-05-19 11:40:12,032 - __main__ - INFO - Running in notebook mode, saving modules...
2025-05-19 11:40:12,034 - __main__ - INFO - Using functions from notebook environment
2025-05-19 11:40:12,035 - __main__ - INFO - Testing one-off chat...
2025-05-19 11:40:12,036 - __main__ - INFO - Testing one-off chat with 5 questions
2025-05-19 11:40:12,036 - __main__ - INFO - Testing question: What are the symptoms of gout?
2025-05-19 11:40:13,548 - __main__ - INFO - Got response: Question: What are the symptoms of gout? Answer: G...
2025-05-19 11:40:13,548 - __main__ - INFO - Testing question: How is gout diagnosed?
2025-05-19 11:40:15,049 - __main__ - INFO - Got response: Question: How is gout diagnosed? Answer: Gout is a...
2025-05-19 11:40:15,050 - __main__ - INFO - Testing question: What treatments are available for gout?
2025-05-19 11:40:16,466 - __main__ - INFO - Got response: Question: What treatments are available for gout?A...
2025-05-19 11:40:16,466 - __main__ - INFO - Testing question

## Progress Checkpoints

1. **API Connection**:
   - [ ] Successfully connect to the Hugging Face API
   - [ ] Send a query and receive a response
   - [ ] Handle API errors gracefully

2. **Chat Function Implementation**:
   - [ ] Implement the get_response function
   - [ ] Create the run_chat function for interactive sessions
   - [ ] Handle errors and edge cases

3. **Command Line Interface**:
   - [ ] Create a parser with appropriate arguments
   - [ ] Implement the main function
   - [ ] Test the CLI functionality

4. **Testing and Evaluation**:
   - [ ] Test the functions with healthcare questions
   - [ ] Save the results in a structured format
   - [ ] Analyze the quality of responses

## Common Issues and Solutions

1. **API Access Issues**:
   - Problem: Rate limiting
   - Solution: Implement exponential backoff and retry logic
   - Problem: Authentication errors
   - Solution: Verify API key and environment variables

2. **Response Parsing Issues**:
   - Problem: Unexpected response format
   - Solution: Add error handling for different response structures
   - Problem: Empty or error responses
   - Solution: Provide meaningful fallback responses

3. **CLI Issues**:
   - Problem: Arguments not parsed correctly
   - Solution: Test with different argument combinations
   - Problem: Script not executable
   - Solution: Check file permissions

## What to Submit

1. Your implementation of the chat scripts:
   - Basic requirement: `utils/one_off_chat.py` for single prompt/response chat
   - Stretch goal (optional): `utils/conversation.py` for contextual chat
   - Testing script: `utils/test_chat.py` to evaluate your implementation

2. Test results in `results/part_2/example.txt` with the following format:
   - Usage examples section showing how to run your scripts
   - Test results section with CSV-formatted question/response pairs
   - If you implemented the stretch goal, include examples of contextual exchanges

The auto-grader should check:
1. That your chat scripts can be executed
2. That they correctly handle the test questions
3. That your results file contains the required sections